## Import libraries

In [21]:
import urllib
import urllib.request

from urllib.parse import urlparse
from bs4 import BeautifulSoup
import requests
import re #regex
from selenium import webdriver
import csv
from csv import DictWriter
import json
from lxml import html
from lxml import etree

import numpy as np
import pandas as pd

import shutil
import tempfile

# To make it more "human", we can randomize the waiting time:
from random import randint
from time import sleep


In [10]:
pages =[]
iterations=list(range(1,13,1))
#assemble urls
for i in iterations:
    month =str(i)
    url="https://www.christies.com/en/results?year=2021&month=" + month
#download html with get request
    response = requests.get(url)
#monitor the status codes for each page 
    print("status=" +str(response.status_code))
#store pages into a list 
    pages.append(response)
#respectful nap time 
    wait_time=randint(1,4)
    print("i will sleep for..." +str(wait_time) +" seconds now")
    sleep(wait_time)

status=200
i will sleep for...3 seconds now
status=200
i will sleep for...4 seconds now
status=200
i will sleep for...3 seconds now
status=200
i will sleep for...3 seconds now
status=200
i will sleep for...4 seconds now
status=200
i will sleep for...3 seconds now
status=200
i will sleep for...3 seconds now
status=200
i will sleep for...1 seconds now
status=200
i will sleep for...3 seconds now
status=200
i will sleep for...1 seconds now
status=200
i will sleep for...1 seconds now
status=200
i will sleep for...1 seconds now


## Let's try with one result page

### Get the soup

In [11]:
# store url in a variable
url ='http://www.christies.com/results/?month=1&year=1998&locations=&scids=&action=paging&initialpageload=false&pg=1'
# download html with a request, check response code 
response=requests.get(url)
response.status_code
# create a beautiful soup
soup = BeautifulSoup(response.content, 'html.parser')

In [12]:
print(soup.prettify())

<!DOCTYPE html>
<html class="no-js" lang="en">
 <head>
  <meta charset="utf-8"/>
  <script type="text/javascript">
   !function(T,l,y){var S=T.location,k="script",D="instrumentationKey",C="ingestionendpoint",I="disableExceptionTracking",E="ai.device.",b="toLowerCase",w="crossOrigin",N="POST",e="appInsightsSDK",t=y.name||"appInsights";(y.name||T[e])&&(T[e]=t);var n=T[t]||function(d){var g=!1,f=!1,m={initialize:!0,queue:[],sv:"5",version:2,config:d};function v(e,t){var n={},a="Browser";return n[E+"id"]=a[b](),n[E+"type"]=a,n["ai.operation.name"]=S&&S.pathname||"_unknown_",n["ai.internal.sdkVersion"]="javascript:snippet_"+(m.sv||m.version),{time:function(){var e=new Date;function t(e){var t=""+e;return 1===t.length&&(t="0"+t),t}return e.getUTCFullYear()+"-"+t(1+e.getUTCMonth())+"-"+t(e.getUTCDate())+"T"+t(e.getUTCHours())+":"+t(e.getUTCMinutes())+":"+t(e.getUTCSeconds())+"."+((e.getUTCMilliseconds()/1e3).toFixed(3)+"").slice(2,5)+"Z"}(),iKey:e,name:"Microsoft.ApplicationInsights."+e.repla

In [13]:
# save prettify txt
with open('url_list_auction.txt', 'w', encoding='utf-8') as f_out:
    f_out.write(soup.prettify())

### Query the soup and get the events links

In [14]:
# select auction links from monthly / year pages
a_tags = soup.find_all("a")
# to get the link we need to: .get("href")
url_list=[]
for a in a_tags:
    urll=a.get("href")
    url_list.append(urll)
print(len(url_list))

url_list = url_list[3:]
url_list = url_list[: len(url_list) - 4] 
print(len(url_list))
url_list

27
20


['https://www.christies.com/FINE-AND-RARE-WINES-5688.aspx?lid=1',
 'https://www.christies.com/OLD-MASTER-DRAWINGS-5542.aspx?lid=1',
 'https://www.christies.com/BRITISH-and-CONTINENTAL-WATERCOLOURS-7310.aspx?lid=1',
 'https://www.christies.com/BRITISH-AND-CONTINENTAL-CERAMICS-7301.aspx?lid=1',
 'https://www.christies.com/CAMERAS-AND-OPTICAL-TOYS-7638.aspx?lid=1',
 'https://www.christies.com/Important-Old-Master-Paintings-5541.aspx?lid=1',
 'https://www.christies.com/MEDICAL-INSTRUMENTS-7309.aspx?lid=1',
 'https://www.christies.com/CARPETS-DECORATIVE-OBJECTS-AND-7306.aspx?lid=1',
 'https://www.christies.com/Decorative-Sporting-Topographical-and-7307.aspx?lid=1',
 'https://www.christies.com/Important-European-Works-of-5540.aspx?lid=1',
 'https://www.christies.com/COLLECTION-OF-LOUIS-VUITTON-7637.aspx?lid=1',
 'https://www.christies.com/COSTUME-AND-TEXTILES-7636.aspx?lid=1',
 'https://www.christies.com/JEWELLERY-7305.aspx?lid=1',
 'https://www.christies.com/ORIENTAL-WORKS-OF-ART-7635.aspx?

In [108]:
# removing header links
#url_list = url_list[3:]
#url_list

In [109]:
# removing footer links
#url_list = url_list[: len(url_list) - 4] 
#url_list

In [110]:
#len(url_list)

## Let's try with the whole results url_list 

### Import the url_list

In [92]:
# until here we have the links of one results page. 
# Now we have to extract the links of all the results pages stored at "url_list.csv"

In [15]:
# read links csv 
url_list = pd.read_csv("url_list.csv") 
url_list

,url
0,http://www.christies.com/results/?month=1&year...
1,http://www.christies.com/results/?month=1&year...
2,http://www.christies.com/results/?month=1&year...
3,http://www.christies.com/results/?month=1&year...
4,http://www.christies.com/results/?month=1&year...
...,...
859,http://www.christies.com/results/?month=12&yea...
860,http://www.christies.com/results/?month=12&yea...
861,http://www.christies.com/results/?month=12&yea...
862,http://www.christies.com/results/?month=12&yea...


In [16]:
all_urls = []
with open('url_list.csv') as csv_file:
    reader = csv.reader(csv_file)
    for row in reader:
        all_urls.append([url])

In [17]:
all_urls

[['http://www.christies.com/results/?month=1&year=1998&locations=&scids=&action=paging&initialpageload=false&pg=1'],
 ['http://www.christies.com/results/?month=1&year=1998&locations=&scids=&action=paging&initialpageload=false&pg=1'],
 ['http://www.christies.com/results/?month=1&year=1998&locations=&scids=&action=paging&initialpageload=false&pg=1'],
 ['http://www.christies.com/results/?month=1&year=1998&locations=&scids=&action=paging&initialpageload=false&pg=1'],
 ['http://www.christies.com/results/?month=1&year=1998&locations=&scids=&action=paging&initialpageload=false&pg=1'],
 ['http://www.christies.com/results/?month=1&year=1998&locations=&scids=&action=paging&initialpageload=false&pg=1'],
 ['http://www.christies.com/results/?month=1&year=1998&locations=&scids=&action=paging&initialpageload=false&pg=1'],
 ['http://www.christies.com/results/?month=1&year=1998&locations=&scids=&action=paging&initialpageload=false&pg=1'],
 ['http://www.christies.com/results/?month=1&year=1998&locations

### Attempt 1

In [159]:
for link in all_urls:
    # download html with a request, check response code 
    response=requests.get(url)
    response.status_code
    sleep(3)
    
    # create a beautiful soup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # save prettify txt
    with open('result_test.txt', 'w', encoding='utf-8') as f_out:
        f_out.write(soup.prettify())
        
    # select auction links from monthly / year pages
    a_tags = soup.find_all("a")
    
    # to get the link we need to: .get("href")
    url_list=[]
    events_list=[]
    
    for a in a_tags:
        urll=a.get("href")
        url_list.append(urll)
        print(len(url_list))
        
        url_list = url_list[3:]
        url_list = url_list[: len(url_list) - 4] 
        print(len(url_list))
        
        events_list.extend(url_list)
        print(len(events_list))
events_list

13
6
6
43
36
42
72
65
107
126
119
226
180
173
399
251
244
643
318
311
954
377
370
1324
442
435
1759
496
489
2248
561
554
2802
633
626
3428
695
688
4116
757
750
4866
812
805
5671
856
849
6520
912
905
7425
967
960
8385
1027
1020
9405
1096
1089
10494
1160
1153
11647
1211
1204
12851
1272
1265
14116
1332
1325
15441
1454
1447
16888
1564
1557
18445
1635
1628
20073
13
6
6
43
36
42
72
65
107
126
119
226
180
173
399
251
244
643
318
311
954
377
370
1324
442
435
1759
496
489
2248
561
554
2802
633
626
3428
695
688
4116
757
750
4866
812
805
5671
856
849
6520
912
905
7425
967
960
8385
1027
1020
9405
1096
1089
10494
1160
1153
11647
1211
1204
12851
1272
1265
14116
1332
1325
15441
1454
1447
16888
1564
1557
18445
1635
1628
20073
13
6
6
43
36
42
72
65
107
126
119
226
180
173
399
251
244
643
318
311
954
377
370
1324
442
435
1759
496
489
2248
561
554
2802
633
626
3428
695
688
4116
757
750
4866
812
805
5671
856
849
6520
912
905
7425
967
960
8385
1027
1020
9405
1096
1089
10494
1160
1153
11647
1211
1204
12851


13
6
6
43
36
42
72
65
107
126
119
226
180
173
399
251
244
643
318
311
954
377
370
1324
442
435
1759
496
489
2248
561
554
2802
633
626
3428
695
688
4116
757
750
4866
812
805
5671
856
849
6520
912
905
7425
967
960
8385
1027
1020
9405
1096
1089
10494
1160
1153
11647
1211
1204
12851
1272
1265
14116
1332
1325
15441
1454
1447
16888
1564
1557
18445
1635
1628
20073
13
6
6
43
36
42
72
65
107
126
119
226
180
173
399
251
244
643
318
311
954
377
370
1324
442
435
1759
496
489
2248
561
554
2802
633
626
3428
695
688
4116
757
750
4866
812
805
5671
856
849
6520
912
905
7425
967
960
8385
1027
1020
9405
1096
1089
10494
1160
1153
11647
1211
1204
12851
1272
1265
14116
1332
1325
15441
1454
1447
16888
1564
1557
18445
1635
1628
20073
13
6
6
43
36
42
72
65
107
126
119
226
180
173
399
251
244
643
318
311
954
377
370
1324
442
435
1759
496
489
2248
561
554
2802
633
626
3428
695
688
4116
757
750
4866
812
805
5671
856
849
6520
912
905
7425
967
960
8385
1027
1020
9405
1096
1089
10494
1160
1153
11647
1211
1204
12851


13
6
6
43
36
42
72
65
107
126
119
226
180
173
399
251
244
643
318
311
954
377
370
1324
442
435
1759
496
489
2248
561
554
2802
633
626
3428
695
688
4116
757
750
4866
812
805
5671
856
849
6520
912
905
7425
967
960
8385
1027
1020
9405
1096
1089
10494
1160
1153
11647
1211
1204
12851
1272
1265
14116
1332
1325
15441
1454
1447
16888
1564
1557
18445
1635
1628
20073
13
6
6
43
36
42
72
65
107
126
119
226
180
173
399
251
244
643
318
311
954
377
370
1324
442
435
1759
496
489
2248
561
554
2802
633
626
3428
695
688
4116
757
750
4866
812
805
5671
856
849
6520
912
905
7425
967
960
8385
1027
1020
9405
1096
1089
10494
1160
1153
11647
1211
1204
12851
1272
1265
14116
1332
1325
15441
1454
1447
16888
1564
1557
18445
1635
1628
20073
13
6
6
43
36
42
72
65
107
126
119
226
180
173
399
251
244
643
318
311
954
377
370
1324
442
435
1759
496
489
2248
561
554
2802
633
626
3428
695
688
4116
757
750
4866
812
805
5671
856
849
6520
912
905
7425
967
960
8385
1027
1020
9405
1096
1089
10494
1160
1153
11647
1211
1204
12851


KeyboardInterrupt: 

In [148]:
events_list
print(len(events_list))

1


In [149]:
events_list

[['https://www.christies.com/FINE-AND-RARE-WINES-5688.aspx?lid=1',
  'https://www.christies.com/OLD-MASTER-DRAWINGS-5542.aspx?lid=1',
  'https://www.christies.com/BRITISH-and-CONTINENTAL-WATERCOLOURS-7310.aspx?lid=1',
  'https://www.christies.com/BRITISH-AND-CONTINENTAL-CERAMICS-7301.aspx?lid=1',
  'https://www.christies.com/CAMERAS-AND-OPTICAL-TOYS-7638.aspx?lid=1',
  'https://www.christies.com/Important-Old-Master-Paintings-5541.aspx?lid=1',
  'https://www.christies.com/MEDICAL-INSTRUMENTS-7309.aspx?lid=1',
  'https://www.christies.com/CARPETS-DECORATIVE-OBJECTS-AND-7306.aspx?lid=1',
  'https://www.christies.com/Decorative-Sporting-Topographical-and-7307.aspx?lid=1',
  'https://www.christies.com/Important-European-Works-of-5540.aspx?lid=1',
  'https://www.christies.com/COLLECTION-OF-LOUIS-VUITTON-7637.aspx?lid=1',
  'https://www.christies.com/COSTUME-AND-TEXTILES-7636.aspx?lid=1',
  'https://www.christies.com/JEWELLERY-7305.aspx?lid=1',
  'https://www.christies.com/ORIENTAL-WORKS-OF-

In [ ]:
# it takes more or less 50 min to get all the results! 

### Attempt 2

In [155]:
for link in all_urls:
    # download html with a request, check response code 
    response=requests.get(url)
    response.status_code
    sleep(3)
    
    # create a beautiful soup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # save prettify txt
    with open('result_test.txt', 'w', encoding='utf-8') as f_out:
        f_out.write(soup.prettify())
        
    # select auction links from monthly / year pages
    a_tags = soup.find_all("a")
    
    # to get the link we need to: .get("href")
    url_list=[]
    events_list=[]
    
    for a in a_tags:
        urll=a.get("href")
        url_list.append(urll)
        print(len(url_list))
        url_list = url_list[3:]
        url_list = url_list[: len(url_list) - 4] 
        print(len(url_list))
        events_list.extend(url_list)
        print(len(events_list))
events_list

1
0
0
1
0
0
1
0
0
1
0
0
1
0
0
1
0
0
1
0
0
1
0
0
1
0
0
1
0
0
1
0
0
1
0
0
1
0
0
1
0
0
1
0
0
1
0
0
1
0
0
1
0
0
1
0
0
1
0
0
1
0
0
1
0
0
1
0
0
1
0
0
1
0
0
1
0
0
1
0
0


KeyboardInterrupt: 

### Attempt 3

In [22]:
for link in all_urls:
    # download html with a request, check response code 
    response=requests.get(url)
    response.status_code
    sleep(3)
    
    # create a beautiful soup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # save prettify txt
    with open('result_test.txt', 'w', encoding='utf-8') as f_out:
        f_out.write(soup.prettify())
        
    # select auction links from monthly / year pages
    a_tags = soup.find_all("a")
    
    # to get the link we need to: .get("href")
    url_list=[]
    events_list=[]
    
    for a in a_tags: 
        urll=a.get("href")
        url_list.append(urll)
        print(len(url_list))
        
        #inner loop for clean each new list 
        for url in url_list:
            url_list = url_list[3:]
            url_list = url_list[: len(url_list) - 4] 
            print(len(url_list))
            
            #inner loop for append the url to the events_list
            for event in events_list:
                events_list.extend(url_list)
                print(len(events_list))
events_list

MissingSchema: Invalid URL '/about-us/contact/privacy/': No schema supplied. Perhaps you meant http:///about-us/contact/privacy/?